In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("janiobachmann/bank-marketing-dataset")

print("Path to dataset files:", path)

/home/cloud/anaconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 142k/142k [00:00<00:00, 335kB/s]

Extracting files...
Path to dataset files: /home/cloud/.cache/kagglehub/datasets/janiobachmann/bank-marketing-dataset/versions/1


In [39]:
!pip install xgboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 MB 169.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.8/289.8 MB 105.2 MB/s eta 0:00:0000:0100:01


In [1]:
import pandas as pd
import numpy as np

# Visualization (optional but useful)
import matplotlib.pyplot as plt
import seaborn as sns

# ML utilities
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import (
    accuracy_score, roc_auc_score, precision_score,
    recall_score, f1_score, matthews_corrcoef,
    confusion_matrix, classification_report
)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [2]:
path = "/home/cloud/.cache/kagglehub/datasets/janiobachmann/bank-marketing-dataset/versions/1/bank.csv"

df = pd.read_csv(path)
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


In [5]:
target = "deposit"

X = df.drop(columns=[target])
y = df[target]

In [7]:
df['deposit'].value_counts()

deposit
no     5873
yes    5289
Name: count, dtype: int64

In [9]:
le_target = LabelEncoder()
y = le_target.fit_transform(y)   

In [11]:
X = pd.get_dummies(X, drop_first=True)

In [13]:
X

,age,balance,day,duration,campaign,pdays,previous,job_blue-collar,job_entrepreneur,job_housemaid,...,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_other,poutcome_success,poutcome_unknown
0,59,2343,5,1042,1,-1,0,False,False,False,...,False,False,False,True,False,False,False,False,False,True
1,56,45,5,1467,1,-1,0,False,False,False,...,False,False,False,True,False,False,False,False,False,True
2,41,1270,5,1389,1,-1,0,False,False,False,...,False,False,False,True,False,False,False,False,False,True
3,55,2476,5,579,1,-1,0,False,False,False,...,False,False,False,True,False,False,False,False,False,True
4,54,184,5,673,2,-1,0,False,False,False,...,False,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,1,20,257,1,-1,0,True,False,False,...,False,False,False,False,False,False,False,False,False,True
11158,39,733,16,83,4,-1,0,False,False,False,...,False,True,False,False,False,False,False,False,False,True
11159,32,29,19,156,2,-1,0,False,False,False,...,False,False,False,False,False,False,False,False,False,True
11160,43,0,8,9,2,172,5,False,False,False,...,False,False,False,True,False,False,False,False,False,False


In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [17]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [19]:
from sklearn.metrics import *

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:,1]

    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "AUC": roc_auc_score(y_test, y_prob),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1": f1_score(y_test, y_pred),
        "MCC": matthews_corrcoef(y_test, y_pred)
    }

In [21]:
print(X.shape)
print(X_train.shape, X_test.shape)

(11162, 42)
(8929, 42) (2233, 42)


In [ ]:
# Take small sample from ORIGINAL dataframe
test_df = df.sample(n=200, random_state=42)

test_df.head()

In [43]:
results = {}

log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train_scaled, y_train)
results["Logistic Regression"] = evaluate_model(
    log_model, X_test_scaled, y_test
)

In [45]:
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)
results["Decision Tree"] = evaluate_model(
    dt_model, X_test, y_test
)


In [47]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_scaled, y_train)
results["KNN"] = evaluate_model(
    knn_model, X_test_scaled, y_test
)

In [49]:
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
results["Naive Bayes"] = evaluate_model(
    nb_model, X_test, y_test
)

In [51]:
rf_model = RandomForestClassifier(
    n_estimators=100,
    random_state=42
)
rf_model.fit(X_train, y_train)
results["Random Forest"] = evaluate_model(
    rf_model, X_test, y_test
)


In [53]:
xgb_model = XGBClassifier(
    eval_metric="logloss",
    use_label_encoder=False,
    random_state=42
)
xgb_model.fit(X_train, y_train)
results["XGBoost"] = evaluate_model(
    xgb_model, X_test, y_test
)

/home/cloud/anaconda3/lib/python3.12/site-packages/xgboost/training.py:200: UserWarning: [16:08:31] WARNING: /__w/xgboost/xgboost/src/learner.cc:782: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [55]:
results_df = pd.DataFrame(results).T
results_df

,Accuracy,AUC,Precision,Recall,F1,MCC
Logistic Regression,0.825795,0.907194,0.826979,0.799622,0.813071,0.650377
Decision Tree,0.792208,0.791212,0.785577,0.772212,0.778837,0.582995
KNN,0.778325,0.841606,0.799787,0.709830,0.752128,0.556148
Naive Bayes,0.733094,0.822651,0.799223,0.583176,0.674317,0.473757
Random Forest,0.862517,0.919235,0.832595,0.888469,0.859625,0.726664
XGBoost,0.864308,0.925727,0.841012,0.879962,0.860046,0.729198


In [57]:
import joblib
import os

os.makedirs("model", exist_ok=True)

joblib.dump(log_model, "model/logistic.pkl")
joblib.dump(dt_model, "model/dt.pkl")
joblib.dump(knn_model, "model/knn.pkl")
joblib.dump(nb_model, "model/nb.pkl")
joblib.dump(rf_model, "model/rf.pkl")
joblib.dump(xgb_model, "model/xgb.pkl")
joblib.dump(scaler, "model/scaler.pkl")

['model/scaler.pkl']